# モジュールインポート

In [1]:
# scikitlearnのアルゴリズムのロード
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier 

# scikit learnの精度測定ツールの読み込み
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# pandas(pd) とNumpy(np)のモジュール
import numpy as np
import pandas as pd
import sys,os,random
import warnings
import pickle

warnings.filterwarnings('ignore')
%matplotlib inline

# 変数定義

In [2]:
f = open("./model/predict_columns.txt","r")
predict_columns = []
for x in f:
    predict_columns.append(x.rstrip("\n"))
f.close()
#predict_columns

FileNotFoundError: [Errno 2] No such file or directory: './model/predict_columns.txt'

# 前処理済みファイルのロード

In [6]:
# 乱数のシード固定
np.random.seed(0)
random.seed(0)
# データフレームの表示件数の設定
pd.set_option('display.max_columns', 100)

# データロード
df = pd.read_pickle('data/df_test.pkl')

y_loc=df.columns.get_loc('result')
#df=df.dropna(how="any",axis=0)

display(df.head(3))

display(df.tail(10))

# 前後のデータのカット
#df=df.loc[NaN_no:(len(df)-NaN_no),:]
df_num=df.as_matrix()   #numpyへ変換

df_len=len(df)

FileNotFoundError: [Errno 2] No such file or directory: 'data/df_test.pkl'

In [5]:
#Numpy形式に変換
# 説明変数
testX=df[predict_columns].as_matrix()
# 正解データ
testY=df.iloc[:,y_loc_s:y_loc_e+1].as_matrix()
testY =np.argmax(testY,axis=1)

#データサンプル数の確認
print("　 　 テストデータ",testX.shape)
print("　 　 テストデータ",testY.shape)

　 　 テストデータ (969, 24)
　 　 テストデータ (969,)


# sckit-learnの準備


In [6]:
# 保存したモデルをロードする
filename = 'model/sk-learn_model.sav'
classifier = pickle.load(open(filename, 'rb'))

In [7]:
df['predict']=np.argmax(classifier.predict_proba(testX),axis=1)
df['predict']=df['predict'].apply(lambda x: "買" if x==0  else "売")
df['result2']=((df['result']-1)*100).round(2)
def check_asnswer(row):
    if row['predict']=="売" and row['result2']<=0:
        return "○"
    if row['predict']=="売" and row['result2']>0:
        return "×"
    if row['predict']=="買" and row['result2']>=0:
        return "○"
    if row['predict']=="買" and row['result2']<0:
        return "×"
    return "-"
df['正誤'] = df.apply(check_asnswer, axis=1)
df['正誤'].value_counts()

AttributeError: predict_proba is not available when  probability=False

# シミュレーション

In [8]:
df_num=df.as_matrix()   #numpyへ変換

In [9]:
close_index=df.columns.get_loc('Close')
open_index=df.columns.get_loc('Open')
date_index=df.columns.get_loc('Date')

MONEY=1000000
LEN=len(df_num)

start=len(df_num)-len(testX)
end=len(df_num)-1
print("start:",df_num[start,0],"〜end:",df_num[end,0])

standard=1000000/df_num[start,1]  
DATE=df['Date'].values[0]
result_out=""
df_sim = pd.DataFrame( columns=['Date', 'Open', 'Close', 'Up', 'money','基準価格','result','predict','正誤','利益率'] )

start: 2016-06-24 00:00:00 〜end: 2020-07-30 00:00:00


In [10]:
# 説明変数
y_loc_s=df.columns.get_loc('label_up')
y_loc_e=df.columns.get_loc('label_down')
y_loc_up=df.columns.get_loc('label_up')
y_loc_down=df.columns.get_loc('label_down')


In [11]:
threshold=0.55
def judgment(PRE2):
    arg=np.argmax(PRE2,axis=0)
    if PRE2[arg]>threshold:
        arg=np.argmax(PRE2,axis=0)
        #print("確率:", PRE2[arg],end=" ")

    else:
        arg=2

    return arg

In [12]:
MONEY=1000000
DATE=""
for i in range(start,end): 
        #PRE=df_num[i,array_list]
        #予測のコマンド
        PRE=df[predict_columns].iloc[i].as_matrix()
        PRE2 =classifier.predict_proba([PRE])[0]
        dision=judgment(PRE2)
        
        profit_rate=0
        DATE2=str(df_num[i,date_index])
         
        if DATE != DATE2[:7]:       #月初の定点観測
            DATE=DATE2[:7]
            #print(DATE,DATE2    )
            print()
            result_out=result_out+"\n"
            if DATE2[5:7]=="01":
                print("--------------------------------------------------------------------")
                result_out=result_out+"--------------------------------------------------------------------"+"\n"
            STRING=DATE2[:7]+":資金"+"{:,}".format(int(MONEY))+"(基準"+"{:,}".format(int(standard*df_num[i,1])).strip()+")"

            
            print(STRING.strip(),end="")
            result_out=result_out+STRING.strip()+"\n"



        PREDICT    ="None"
        
        #  買い予測の場合
        if dision==0:
            PREDICT="buy"
            print("買",end=" ")
            profit_rate=df_num[i,close_index]/df_num[i,open_index]
            MONEY=MONEY*profit_rate

        #  売り予測の場合
        if dision==1:
            PREDICT="sell"
            print("売",end=" ")
            profit_rate=df_num[i,close_index]/df_num[i,open_index]
            MONEY=MONEY/profit_rate

        if df_num[i,y_loc_up]==1:
            Y_result="buy"
        elif df_num[i,y_loc_down]==1:
            Y_result="sell"
        else:
            Y_result="None"
            
        # 正誤表の処理
        seigo="-"    
        #print("正誤表",df_num[i,66],Y_result,PREDICT)
        if Y_result==PREDICT:
            seigo="●"
        else:
            seigo="×"    

        
        if profit_rate!=0:
            profit_rate=100*(profit_rate-1)
        
        s1 = pd.Series([df_num[i,0],                 #日付
                        round(df_num[i,open_index], 2),        #OPEN
                        round(df_num[i,close_index],2),        #Close
                        round((df_num[i,close_index]/df_num[i,open_index]-1)*100,2),  #アップ率
                        round(MONEY),
                        int(standard*df_num[i,1]),                 #基準価格
                        Y_result,PREDICT,seigo,
                        round(profit_rate,2)],                     #予測
               index=df_sim.columns)
        df_sim=df_sim.append(s1,ignore_index=True)

AttributeError: predict_proba is not available when  probability=False

In [13]:
df_sim[(
    df_sim['Date'].str.contains('01-01') |
    df_sim['Date'].str.contains('04-01') |
    df_sim['Date'].str.contains('07-01') |
    df_sim['Date'].str.contains('10-01') 
)]

,Date,Open,Close,Up,money,基準価格,result,predict,正誤,利益率


In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

df_sim.plot(x='Date',y='money',title='Money/Date')

TypeError: no numeric data to plot

In [16]:
print("●",df_sim[df_sim['正誤']=="●"].shape[0],"回 勝ち")
print("X",df_sim[df_sim['正誤']=="×"].shape[0],"回 負け")

for i in range(7,21):
    YEAR=str(i+2000)
    start_day=YEAR+"-01-01"
    end_day=YEAR+"-12-31"

    tmp=df_sim.query("Date >= @start_day and Date <= @end_day ")
    
    print("{:}年の利益率{:.2f}%".format(YEAR,tmp[['利益率']].sum()[0]))

● 285 回 勝ち
X 671 回 負け
2007年の利益率0.00%
2008年の利益率0.00%
2009年の利益率0.00%
2010年の利益率0.00%
2011年の利益率0.00%
2012年の利益率0.00%
2013年の利益率0.00%
2014年の利益率0.00%
2015年の利益率0.00%
2016年の利益率-3.24%
2017年の利益率-1.51%
2018年の利益率-12.87%
2019年の利益率4.84%
2020年の利益率2.17%
